In [17]:
import sklearn
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
import spacy

from conllu_to_df import process_file
from extract_features import extract_features

In [18]:
train_file = "../UP-1.0/input/en_ewt-up-train.conllu"
dev_file = "../UP-1.0/input/en_ewt-up-dev.conllu"
test_file = "../UP-1.0/input/en_ewt-up-test.conllu"

out_train_file = '../UP-1.0/output/train.csv'
out_test_file = '../UP-1.0/output/test.csv'
out_dev_file = '../UP-1.0/output/dev.csv'

feat_train_file = out_train_file.replace('output', 'features')
feat_test_file = out_test_file.replace('output', 'features')
feat_dev_file = out_dev_file.replace('output', 'features')

In [19]:
process_file(dev_file, out_dev_file)
process_file(test_file, out_test_file)
process_file(train_file, out_train_file)

105042
100928
1023133


In [20]:
extract_features(out_dev_file, feat_dev_file)
extract_features(out_train_file, feat_train_file)
extract_features(out_test_file, feat_test_file)

In [21]:
vectorizer = DictVectorizer()

In [23]:
# dev only for now
dev_df = pd.read_csv(feat_dev_file, sep='\t')
dev_data = dev_df.to_dict('records')
del dev_df
dev_vectorized = vectorizer.fit_transform(dev_data)

In [24]:
dev_vectorized.shape

(106023, 12493)

In [ ]:
target_col = 'some_column_name'

X = train_data.loc[:, train_data.columns != target_col]
X_dev = dev_data.loc[:, dev_data.columns != target_col]

y = train_data.target_col
y_dev = dev_data.target_col

In [ ]:
model = LogisticRegression().fit(X, y)

In [ ]:
y_pred = model.predict(X_dev)

In [ ]:
print("METRICS")
print('precision', precision_score(y_dev, y_pred), sep='\t')
print('recall', recall_score(y_dev, y_pred), sep='\t')
print('f1', f1_score(y_dev, y_pred), sep='\t')